<a href="https://colab.research.google.com/github/leopomme/Dose-Prediction-Segmentation/blob/main/tensorflow_2_pytorch_DLMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import torch
import cv2
from PIL import Image
import torchvision.transforms as transforms
from torchvision.models import resnet50
from keras.layers.convolutional import Conv2D
import keras
import collections

inputpath = '/content/drive/MyDrive/RadImageNet-ResNet50_notop.h5'
outpath = '/content/drive/MyDrive/RadImageNet-ResNet50_notop_torch.pth'
testimg = '/content/drive/MyDrive/img_test_drive.png'

def simple_test(net):
    img = Image.open(testimg).convert('L')  # Change 'RGB' to 'L' for grayscale

    trans_data = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
    ])

    img = trans_data(img).unsqueeze(0)
    img_rgb = img.repeat(1, 3, 1, 1)  # Repeat the single channel 3 times to create an RGB image with dimensions [N, C, H, W] = [1, 3, H, W]
    print(img_rgb.shape)
    out = net(img_rgb)
    return out.squeeze(0)[0]


def keras_to_pyt(km, pm=None):
  weight_dict = dict()
  
  for layer in km.layers:
      layer_name = layer.get_config()['name']
      
      if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
          weight_dict[layer_name + '.weight'] = np.transpose(layer.get_weights()[0], (3, 2, 0, 1)) if len(layer.get_weights()[0].shape) == 4 else np.transpose(layer.get_weights()[0], (1, 0))
          weight_dict[layer_name + '.bias'] = layer.get_weights()[1]
      elif isinstance(layer, keras.layers.BatchNormalization):
          weight_dict[layer_name + '.weight'] = layer.get_weights()[0]
          weight_dict[layer_name + '.bias'] = layer.get_weights()[1]
          weight_dict[layer_name + '.running_mean'] = layer.get_weights()[2]
          weight_dict[layer_name + '.running_var'] = layer.get_weights()[3]
  
  if pm:
      pyt_state_dict = pm.state_dict()
      
      for key in pyt_state_dict.keys():
          if key in weight_dict:
              pyt_state_dict[key] = torch.from_numpy(weight_dict[key])
      
      pm.load_state_dict(pyt_state_dict)
      return pm
  
  return weight_dict


net = resnet50(num_classes=1)

out = simple_test(net)
print('before output is', out)

tf_keras_model = tf.keras.models.load_model(inputpath)
weights = keras_to_pyt(tf_keras_model, net)

out = simple_test(net)
print('after output is', out)

torch.save(net.state_dict(), outpath)

torch.Size([1, 3, 224, 224])
before output is tensor(-0.1847, grad_fn=<SelectBackward0>)


torch.Size([1, 3, 224, 224])
after output is tensor(-0.1847, grad_fn=<SelectBackward0>)


In [ ]:
! pip install tf2onnx -qq
! pip install onnx2torch -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.3/442.3 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 KB 11.8 MB/s eta 0:00:00


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
# import onnxruntime

from sklearn.metrics import mean_squared_error as MSE

In [ ]:
# load the tensorflow radimagenet model
model_name = "RadImageNet-ResNet50_notop"
model = tf.keras.models.load_model("/content/drive/MyDrive/RadImageNet-ResNet50_notop.h5")
model.save(os.path.join("/content/drive/MyDrive", model_name))

In [ ]:
# predict using tf radimagenet
a = np.ones((1, 224, 224, 3))
a = tf.convert_to_tensor(a, dtype=tf.float32)
tf_out = model.predict(a)
tf_out = np.array(tf_out)

1/1 [==============================] - 2s 2s/step


In [ ]:
# convert tensorflow model to onnx model
! python -m tf2onnx.convert --opset 13 \
    --saved-model {os.path.join("/content/drive/MyDrive", model_name)} \
    --output  {os.path.join("/content/drive/MyDrive", model_name + ".onnx")}

2023-03-22 13:24:21.236089: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-22 13:24:21.236217: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-22 13:24:21.236243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/

In [ ]:
# convert onnx to pytorch model
import torch
from onnx2torch import convert

onnx_model_path = os.path.join("/content/drive/MyDrive", model_name + ".onnx")
torch_model = convert(onnx_model_path)
torch.save(torch_model.state_dict(), model_name + ".pth")

In [ ]:
# predict using pytorch radimagenet
a = np.ones((1, 224, 224, 3))
a = torch.tensor(a).float()
torch_out = torch_model(a)
torch_out = torch_out.detach().numpy()

In [ ]:
# compare the outputs
print(MSE(tf_out.reshape(-1), torch_out.reshape(-1)))

1.6138929e-12
